# Model Training (Deep Learning)

We train two deep learning models (ANN and 1D-CNN) on each of the nine feature-selection datasets.

For each method and model we:
- Train on 80% train split and evaluate on the provided test split.
- Compute Accuracy, Precision, Recall, F1.
- Save the Confusion Matrix (PNG) and Accuracy vs Epoch plot (PNG).
- Save the trained model (.h5).
- Save results CSV

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import models, layers, callbacks
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

FEATURES_BASE = Path("../data/processed/features2")
PROC_BASE = Path("../data/processed/dl2")
MODEL_BASE = Path("../models/dl2")
FIG_BASE = Path("../figures/dl2")

for p in [PROC_BASE, MODEL_BASE, FIG_BASE]:
    p.mkdir(parents=True, exist_ok=True)

METHODS = ["rfe","skb","fscs","etc","pc","mi","mir","mu","vt"]
RANDOM_STATE = 42
tf.random.set_seed(RANDOM_STATE)

EPOCHS = 50
BATCH_SIZE = 32
VERBOSE = 0

## Model Architectures

In [ ]:
def build_ann(input_dim, num_classes):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn(input_dim, num_classes):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Reshape((input_dim, 1)),
        layers.Conv1D(64, 3, activation='relu', padding='same'),
        layers.MaxPooling1D(2),
        layers.Conv1D(32, 3, activation='relu', padding='same'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## Helper Functions

In [ ]:
def save_confusion(y_true, y_pred, path, title):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_title(title); ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    fig.savefig(path, dpi=300, bbox_inches="tight"); plt.close(fig)

def save_accuracy_plot(history, path, title):
    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(history.history.get('accuracy', []), label='Train Acc')
    ax.plot(history.history.get('val_accuracy', []), label='Val Acc')
    ax.set_xlabel("Epoch"); ax.set_ylabel("Accuracy"); ax.set_title(title); ax.legend()
    fig.savefig(path, dpi=300, bbox_inches="tight"); plt.close(fig)

def compute_metrics(y_true, y_pred):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="weighted", zero_division=0),
        "Recall": recall_score(y_true, y_pred, average="weighted", zero_division=0),
        "F1": f1_score(y_true, y_pred, average="weighted", zero_division=0)
    }

## Model Training Function

In [ ]:
def train_and_save(method, model_type, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE):
    in_dir = FEATURES_BASE / method
    if not in_dir.exists():
        raise FileNotFoundError(f"Missing: {in_dir}")

    train_df = pd.read_csv(in_dir / "train.csv").dropna(subset=["DepressionEncoded"])
    test_df = pd.read_csv(in_dir / "test.csv").dropna(subset=["DepressionEncoded"])

    X_train = train_df.drop(columns=["DepressionEncoded"]).values.astype(np.float32)
    y_train = train_df["DepressionEncoded"].astype(int).values
    X_test = test_df.drop(columns=["DepressionEncoded"]).values.astype(np.float32)
    y_test = test_df["DepressionEncoded"].astype(int).values

    num_classes = len(np.unique(y_train))
    y_train_oh = to_categorical(y_train, num_classes=num_classes)
    y_test_oh = to_categorical(y_test, num_classes=num_classes)

    input_dim = X_train.shape[1]
    model = build_ann(input_dim, num_classes) if model_type == "ANN" else build_cnn(input_dim, num_classes)

    es = callbacks.EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True, verbose=0)
    history = model.fit(X_train, y_train_oh, validation_split=0.2, epochs=epochs, batch_size=batch_size, callbacks=[es], verbose=verbose)

    y_pred = np.argmax(model.predict(X_test), axis=1)
    metrics = compute_metrics(y_test, y_pred)

    proc_out = PROC_BASE / method; model_out = MODEL_BASE / method; fig_out = FIG_BASE / method
    for p in [proc_out, model_out, fig_out]:
        p.mkdir(parents=True, exist_ok=True)

    model_file = model_out / f"{model_type.lower()}_model.h5"
    model.save(model_file)

    save_confusion(y_test, y_pred, fig_out / f"{model_type.lower()}_confusion.png", f"{model_type} Confusion ({method})")
    save_accuracy_plot(history, fig_out / f"{model_type.lower()}_accuracy.png", f"{model_type} Accuracy vs Epoch ({method})")

    res_file = proc_out / "results_deep_learning.csv"
    row = pd.DataFrame([{"Method": method, "Model": model_type, **metrics}])
    if res_file.exists():
        row.to_csv(res_file, mode='a', header=False, index=False)
    else:
        row.to_csv(res_file, index=False)

    return metrics

## Run ANN and CNN for All Feature Sets

In [ ]:
results_all = []
for method in METHODS:
    print("\n" + "="*60)
    print(f"▶ DL training for: {method.upper()}")
    for model_type in ["ANN","CNN"]:
        try:
            print(f" - Training {model_type}")
            metrics = train_and_save(method, model_type)
            results_all.append({"Method": method, "Model": model_type, **metrics})
        except Exception as e:
            print(f"⚠️ Error for {method} {model_type}: {e}")

combined = pd.DataFrame(results_all)
combined_out = PROC_BASE / "all_deep_learning_results_summary_v3.csv"
combined.to_csv(combined_out, index=False)
print("✅ DL combined summary saved to:", combined_out)
display(combined)